<div align="center">

# **Predicciones de Modelos GLM**

</div>

Este notebook calcula las predicciones para todas las combinaciones posibles de variables explicativas en los modelos de frecuencia y severidad.

## Librerías y Carga de Modelos

In [1]:
library(dplyr)
library(tidyr)
library(knitr)
library(kableExtra)

# Cargar modelos entrenados
modelo_frecuencia <- readRDS("../models/modelo_frecuencia_quasipoisson.rds")
modelo_severidad <- readRDS("../models/modelo_severidad_lognormal.rds")

# Cargar datos procesados
datos <- read.csv("../data/processed/tabla_severidad.csv")
datos$X <- NULL


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows




## Establecer Categorías de Referencia

In [2]:
establecer_categorias_referencia <- function(df, vars_categoricas, var_exposicion = "exposicion_total") {
  df_copy <- df
  
  for (var in vars_categoricas) {
    if (var %in% colnames(df_copy)) {
      exposicion_por_categoria <- aggregate(df_copy[[var_exposicion]], 
                                          by = list(df_copy[[var]]), 
                                          FUN = sum, na.rm = TRUE)
      names(exposicion_por_categoria) <- c("categoria", "exposicion_total")
      
      categoria_referencia <- exposicion_por_categoria$categoria[which.max(exposicion_por_categoria$exposicion_total)]
      
      df_copy[[var]] <- factor(df_copy[[var]])
      df_copy[[var]] <- relevel(df_copy[[var]], ref = as.character(categoria_referencia))
    }
  }
  
  return(df_copy)
}

vars_categoricas <- c("Modelo", "Color", "Carroceria", "SERVICIO", "Sexo_Aseg", "Edad", "Gama")
datos <- establecer_categorias_referencia(datos, vars_categoricas)

# Verificar niveles de factores
cat("Niveles de variables:\n")
cat("Modelo:", levels(datos$Modelo), "\n")
cat("Edad:", levels(datos$Edad), "\n")
cat("Color:", levels(datos$Color), "\n")
cat("Carroceria:", levels(datos$Carroceria), "\n")
cat("SERVICIO:", levels(datos$SERVICIO), "\n")

Niveles de variables:
Modelo: 2007_2013 1993_1999 2000_2006 
Edad: 41_63 19_41 63_85 Otros 
Color: Otros AMARILLO BLANCO PLATA 
Carroceria: SEDAN CABINADO HATCHBACK Otros 
SERVICIO: Particular Oficial Publico Transporte Personal 


## Predicciones del Modelo de Frecuencia

Variables del modelo: **Modelo + Edad**

In [3]:
# Crear todas las combinaciones posibles para frecuencia
combinaciones_freq <- expand.grid(
  Modelo = levels(datos$Modelo),
  Edad = levels(datos$Edad),
  stringsAsFactors = FALSE
)

# Añadir exposición estándar de 1 año
combinaciones_freq$exposicion_total <- 1

cat("Combinaciones para modelo de frecuencia:", nrow(combinaciones_freq), "\n")
print(combinaciones_freq)

Combinaciones para modelo de frecuencia: 12 
      Modelo  Edad exposicion_total
1  2007_2013 41_63                1
2  1993_1999 41_63                1
3  2000_2006 41_63                1
4  2007_2013 19_41                1
5  1993_1999 19_41                1
6  2000_2006 19_41                1
7  2007_2013 63_85                1
8  1993_1999 63_85                1
9  2000_2006 63_85                1
10 2007_2013 Otros                1
11 1993_1999 Otros                1
12 2000_2006 Otros                1


In [4]:
# Convertir a factores con los mismos niveles que los datos originales
combinaciones_freq$Modelo <- factor(combinaciones_freq$Modelo, levels = levels(datos$Modelo))
combinaciones_freq$Edad <- factor(combinaciones_freq$Edad, levels = levels(datos$Edad))

# Calcular predicciones de frecuencia
predicciones_freq <- predict(modelo_frecuencia, newdata = combinaciones_freq, type = "response")

# Crear tabla de resultados
resultados_frecuencia <- data.frame(
  Modelo = combinaciones_freq$Modelo,
  Edad = combinaciones_freq$Edad,
  Exposicion = combinaciones_freq$exposicion_total,
  Frecuencia_Esperada = round(predicciones_freq, 6),
  Tasa_Frecuencia = round(predicciones_freq / combinaciones_freq$exposicion_total, 6)
)

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("                    PREDICCIONES DE FRECUENCIA                     \n")
cat("═══════════════════════════════════════════════════════════════════\n")
print(resultados_frecuencia, row.names = FALSE)


═══════════════════════════════════════════════════════════════════
                    PREDICCIONES DE FRECUENCIA                     
═══════════════════════════════════════════════════════════════════
    Modelo  Edad Exposicion Frecuencia_Esperada Tasa_Frecuencia
 2007_2013 41_63          1            0.207627        0.207627
 1993_1999 41_63          1            0.224196        0.224196
 2000_2006 41_63          1            0.269351        0.269351
 2007_2013 19_41          1            0.242110        0.242110
 1993_1999 19_41          1            0.261431        0.261431
 2000_2006 19_41          1            0.314085        0.314085
 2007_2013 63_85          1            0.237058        0.237058
 1993_1999 63_85          1            0.255976        0.255976
 2000_2006 63_85          1            0.307532        0.307532
 2007_2013 Otros          1            0.247011        0.247011
 1993_1999 Otros          1            0.266723        0.266723
 2000_2006 Otros          1

## Predicciones del Modelo de Severidad

Variables del modelo: **Modelo + Color + Carroceria + SERVICIO + Edad**

In [5]:
# Crear todas las combinaciones posibles para severidad
combinaciones_sev <- expand.grid(
  Modelo = levels(datos$Modelo),
  Color = levels(datos$Color),
  Carroceria = levels(datos$Carroceria),
  SERVICIO = levels(datos$SERVICIO),
  Edad = levels(datos$Edad),
  stringsAsFactors = FALSE
)

cat("Combinaciones para modelo de severidad:", nrow(combinaciones_sev), "\n")
cat("Primeras 10 combinaciones:\n")
print(head(combinaciones_sev, 10))

Combinaciones para modelo de severidad: 768 
Primeras 10 combinaciones:
      Modelo    Color Carroceria   SERVICIO  Edad
1  2007_2013    Otros      SEDAN Particular 41_63
2  1993_1999    Otros      SEDAN Particular 41_63
3  2000_2006    Otros      SEDAN Particular 41_63
4  2007_2013 AMARILLO      SEDAN Particular 41_63
5  1993_1999 AMARILLO      SEDAN Particular 41_63
6  2000_2006 AMARILLO      SEDAN Particular 41_63
7  2007_2013   BLANCO      SEDAN Particular 41_63
8  1993_1999   BLANCO      SEDAN Particular 41_63
9  2000_2006   BLANCO      SEDAN Particular 41_63
10 2007_2013    PLATA      SEDAN Particular 41_63


In [6]:
# Convertir a factores con los mismos niveles que los datos originales
combinaciones_sev$Modelo <- factor(combinaciones_sev$Modelo, levels = levels(datos$Modelo))
combinaciones_sev$Color <- factor(combinaciones_sev$Color, levels = levels(datos$Color))
combinaciones_sev$Carroceria <- factor(combinaciones_sev$Carroceria, levels = levels(datos$Carroceria))
combinaciones_sev$SERVICIO <- factor(combinaciones_sev$SERVICIO, levels = levels(datos$SERVICIO))
combinaciones_sev$Edad <- factor(combinaciones_sev$Edad, levels = levels(datos$Edad))

# Calcular predicciones de severidad (en escala logarítmica)
predicciones_sev_log <- predict(modelo_severidad, newdata = combinaciones_sev, type = "response")

# Convertir de logaritmo a escala original
predicciones_sev <- exp(predicciones_sev_log)

# Crear tabla de resultados
resultados_severidad <- data.frame(
  Modelo = combinaciones_sev$Modelo,
  Color = combinaciones_sev$Color,
  Carroceria = combinaciones_sev$Carroceria,
  SERVICIO = combinaciones_sev$SERVICIO,
  Edad = combinaciones_sev$Edad,
  Log_Severidad = round(predicciones_sev_log, 6),
  Severidad_Esperada = round(predicciones_sev, 0)
)

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("                    PREDICCIONES DE SEVERIDAD                      \n")
cat("═══════════════════════════════════════════════════════════════════\n")
cat("Total de predicciones:", nrow(resultados_severidad), "\n\n")
cat("Primeras 20 predicciones:\n")
print(head(resultados_severidad, 20), row.names = FALSE)


═══════════════════════════════════════════════════════════════════
                    PREDICCIONES DE SEVERIDAD                      
═══════════════════════════════════════════════════════════════════
Total de predicciones: 768 

Primeras 20 predicciones:
    Modelo    Color Carroceria   SERVICIO  Edad Log_Severidad
 2007_2013    Otros      SEDAN Particular 41_63      16.86328
 1993_1999    Otros      SEDAN Particular 41_63      16.01421
 2000_2006    Otros      SEDAN Particular 41_63      16.18586
 2007_2013 AMARILLO      SEDAN Particular 41_63      14.59473
 1993_1999 AMARILLO      SEDAN Particular 41_63      13.74566
 2000_2006 AMARILLO      SEDAN Particular 41_63      13.91731
 2007_2013   BLANCO      SEDAN Particular 41_63      16.15380
 1993_1999   BLANCO      SEDAN Particular 41_63      15.30473
 2000_2006   BLANCO      SEDAN Particular 41_63      15.47638
 2007_2013    PLATA      SEDAN Particular 41_63      16.28028
 1993_1999    PLATA      SEDAN Particular 41_63      15.43

## Tabla Pivot de Frecuencia por Modelo y Edad

In [7]:
# Crear tabla pivot para frecuencia
tabla_pivot_frecuencia <- resultados_frecuencia %>%
  select(Modelo, Edad, Tasa_Frecuencia) %>%
  pivot_wider(names_from = Edad, values_from = Tasa_Frecuencia) %>%
  mutate_if(is.numeric, ~ round(.x, 6))

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("                  TABLA PIVOT - FRECUENCIAS                        \n")
cat("═══════════════════════════════════════════════════════════════════\n")
print(tabla_pivot_frecuencia, row.names = FALSE)


═══════════════════════════════════════════════════════════════════
                  TABLA PIVOT - FRECUENCIAS                        
═══════════════════════════════════════════════════════════════════
# A tibble: 3 × 5
  Modelo    `41_63` `19_41` `63_85` Otros
  <fct>       <dbl>   <dbl>   <dbl> <dbl>
1 2007_2013   0.208   0.242   0.237 0.247
2 1993_1999   0.224   0.261   0.256 0.267
3 2000_2006   0.269   0.314   0.308 0.320


## Resumen de Severidad por Combinación de Variables

In [8]:
# Estadísticas descriptivas de severidad
cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("                 ESTADÍSTICAS DE SEVERIDAD                         \n")
cat("═══════════════════════════════════════════════════════════════════\n")
cat("Severidad mínima:", format(min(resultados_severidad$Severidad_Esperada), big.mark = ","), "COP\n")
cat("Severidad máxima:", format(max(resultados_severidad$Severidad_Esperada), big.mark = ","), "COP\n")
cat("Severidad promedio:", format(round(mean(resultados_severidad$Severidad_Esperada)), big.mark = ","), "COP\n")
cat("Factor de variación:", round(max(resultados_severidad$Severidad_Esperada) / min(resultados_severidad$Severidad_Esperada), 2), "veces\n")

# Top 10 severidades más altas
cat("\nTOP 10 SEVERIDADES MÁS ALTAS:\n")
top_severidad <- resultados_severidad %>%
  arrange(desc(Severidad_Esperada)) %>%
  head(10)
print(top_severidad, row.names = FALSE)

# Top 10 severidades más bajas
cat("\nTOP 10 SEVERIDADES MÁS BAJAS:\n")
bottom_severidad <- resultados_severidad %>%
  arrange(Severidad_Esperada) %>%
  head(10)
print(bottom_severidad, row.names = FALSE)


═══════════════════════════════════════════════════════════════════


                 ESTADÍSTICAS DE SEVERIDAD                         
═══════════════════════════════════════════════════════════════════
Severidad mínima: 36,772 COP
Severidad máxima: 168,558,185 COP
Severidad promedio: 10,518,147 COP
Factor de variación: 4583.87 veces

TOP 10 SEVERIDADES MÁS ALTAS:
    Modelo  Color Carroceria SERVICIO  Edad Log_Severidad Severidad_Esperada
 2007_2013  Otros  HATCHBACK  Publico 19_41      18.94279          168558185
 2007_2013  Otros  HATCHBACK  Publico 41_63      18.90057          161588960
 2007_2013  Otros      SEDAN  Publico 19_41      18.77242          142153897
 2007_2013  Otros      SEDAN  Publico 41_63      18.73020          136276387
 2007_2013  Otros  HATCHBACK  Publico 63_85      18.38167           96173907
 2007_2013  Otros  HATCHBACK  Publico Otros      18.37334           95376310
 2007_2013  PLATA  HATCHBACK  Publico 19_41      18.35979           94092616
 2007_2013  PLATA  HATCHBACK  Publico 41_63      18.31756           90202253
 2000_2

## Cálculo de Primas Puras

Prima Pura = Frecuencia × Severidad (para combinaciones compatibles)

In [9]:
# Para calcular primas puras, necesitamos combinar frecuencia y severidad
# pero solo para las combinaciones donde coincidan Modelo y Edad

# Crear combinaciones completas para prima pura
combinaciones_prima <- expand.grid(
  Modelo = levels(datos$Modelo),
  Color = levels(datos$Color),
  Carroceria = levels(datos$Carroceria),
  SERVICIO = levels(datos$SERVICIO),
  Edad = levels(datos$Edad),
  stringsAsFactors = FALSE
)

# Obtener frecuencia para cada combinación (basada en Modelo y Edad)
combinaciones_prima <- combinaciones_prima %>%
  left_join(resultados_frecuencia %>% select(Modelo, Edad, Tasa_Frecuencia), 
            by = c("Modelo", "Edad"))

# Obtener severidad para cada combinación completa
combinaciones_prima <- combinaciones_prima %>%
  left_join(resultados_severidad %>% select(Modelo, Color, Carroceria, SERVICIO, Edad, Severidad_Esperada), 
            by = c("Modelo", "Color", "Carroceria", "SERVICIO", "Edad"))

# Calcular prima pura
combinaciones_prima$Prima_Pura <- combinaciones_prima$Tasa_Frecuencia * combinaciones_prima$Severidad_Esperada

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("                    CÁLCULO DE PRIMAS PURAS                        \n")
cat("═══════════════════════════════════════════════════════════════════\n")
cat("Total de combinaciones:", nrow(combinaciones_prima), "\n")
cat("Prima pura mínima:", format(round(min(combinaciones_prima$Prima_Pura, na.rm = TRUE)), big.mark = ","), "COP\n")
cat("Prima pura máxima:", format(round(max(combinaciones_prima$Prima_Pura, na.rm = TRUE)), big.mark = ","), "COP\n")
cat("Factor de variación:", round(max(combinaciones_prima$Prima_Pura, na.rm = TRUE) / min(combinaciones_prima$Prima_Pura, na.rm = TRUE), 2), "veces\n")

# Mostrar algunas combinaciones ejemplo
cat("\nEjemplos de primas puras:\n")
ejemplos_prima <- combinaciones_prima %>%
  arrange(desc(Prima_Pura)) %>%
  select(Modelo, Color, Carroceria, SERVICIO, Edad, Tasa_Frecuencia, Severidad_Esperada, Prima_Pura) %>%
  mutate(
    Tasa_Frecuencia = round(Tasa_Frecuencia, 6),
    Severidad_Esperada = round(Severidad_Esperada, 0),
    Prima_Pura = round(Prima_Pura, 0)
  ) %>%
  head(15)

print(ejemplos_prima, row.names = FALSE)


═══════════════════════════════════════════════════════════════════
                    CÁLCULO DE PRIMAS PURAS                        
═══════════════════════════════════════════════════════════════════
Total de combinaciones: 768 
Prima pura mínima: 9,492 COP
Prima pura máxima: 40,809,622 COP
Factor de variación: 4299.56 veces

Ejemplos de primas puras:
    Modelo  Color Carroceria SERVICIO  Edad Tasa_Frecuencia Severidad_Esperada
 2007_2013  Otros  HATCHBACK  Publico 19_41        0.242110          168558185
 2007_2013  Otros      SEDAN  Publico 19_41        0.242110          142153897
 2007_2013  Otros  HATCHBACK  Publico 41_63        0.207627          161588960
 2007_2013  Otros      SEDAN  Publico 41_63        0.207627          136276387
 2000_2006  Otros  HATCHBACK  Publico 19_41        0.314085           85615135
 2007_2013  Otros  HATCHBACK  Publico Otros        0.247011           95376310
 2007_2013  Otros  HATCHBACK  Publico 63_85        0.237058           96173907
 2007_201

## Exportar Resultados

In [10]:
# Exportar tablas para uso posterior
write.csv(resultados_frecuencia, "../data/output/predicciones_frecuencia.csv", row.names = FALSE)
write.csv(resultados_severidad, "../data/output/predicciones_severidad.csv", row.names = FALSE)
write.csv(combinaciones_prima, "../data/output/predicciones_prima_pura.csv", row.names = FALSE)

# Crear lista con todos los resultados para LaTeX
resultados_exportar <- list(
  frecuencia = resultados_frecuencia,
  severidad = resultados_severidad,
  prima_pura = combinaciones_prima,
  tabla_pivot_frecuencia = tabla_pivot_frecuencia,
  fecha_calculo = Sys.Date()
)

saveRDS(resultados_exportar, "../data/output/resultados_predicciones_completas.rds")

cat("\n✅ Resultados exportados exitosamente\n")
cat("📁 Archivos generados:\n")
cat("   - predicciones_frecuencia.csv\n")
cat("   - predicciones_severidad.csv\n")
cat("   - predicciones_prima_pura.csv\n")
cat("   - resultados_predicciones_completas.rds\n")


✅ Resultados exportados exitosamente
📁 Archivos generados:


   - predicciones_frecuencia.csv
   - predicciones_severidad.csv
   - predicciones_prima_pura.csv
   - resultados_predicciones_completas.rds


In [11]:
# Generar tabla completa de severidad ordenada por severidad esperada
tabla_severidad_completa <- resultados_severidad %>%
  arrange(desc(Severidad_Esperada)) %>%
  select(Modelo, Color, Carroceria, SERVICIO, Edad, Severidad_Esperada) %>%
  mutate(
    Severidad_Formateada = paste0("\\$", format(Severidad_Esperada, big.mark = ","))
  )

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("           TABLA COMPLETA DE SEVERIDAD (768 COMBINACIONES)         \n")
cat("═══════════════════════════════════════════════════════════════════\n")
cat("Ordenada de mayor a menor severidad esperada\n\n")

# Mostrar primeras 30 filas como ejemplo
cat("Primeras 30 combinaciones (mayor severidad):\n")
print(head(tabla_severidad_completa, 30), row.names = FALSE)

cat("\n...\n")
cat("Últimas 10 combinaciones (menor severidad):\n")
print(tail(tabla_severidad_completa, 10), row.names = FALSE)

# Exportar tabla completa para LaTeX
write.csv(tabla_severidad_completa, "../data/output/tabla_severidad_completa_latex.csv", row.names = FALSE)

cat("\n✅ Tabla completa de severidad exportada: tabla_severidad_completa_latex.csv\n")
cat("📊 Total de combinaciones:", nrow(tabla_severidad_completa), "\n")


═══════════════════════════════════════════════════════════════════
           TABLA COMPLETA DE SEVERIDAD (768 COMBINACIONES)         
═══════════════════════════════════════════════════════════════════
Ordenada de mayor a menor severidad esperada

Primeras 30 combinaciones (mayor severidad):
    Modelo  Color Carroceria SERVICIO  Edad Severidad_Esperada
 2007_2013  Otros  HATCHBACK  Publico 19_41          168558185
 2007_2013  Otros  HATCHBACK  Publico 41_63          161588960
 2007_2013  Otros      SEDAN  Publico 19_41          142153897
 2007_2013  Otros      SEDAN  Publico 41_63          136276387
 2007_2013  Otros  HATCHBACK  Publico 63_85           96173907
 2007_2013  Otros  HATCHBACK  Publico Otros           95376310
 2007_2013  PLATA  HATCHBACK  Publico 19_41           94092616
 2007_2013  PLATA  HATCHBACK  Publico 41_63           90202253
 2000_2006  Otros  HATCHBACK  Publico 19_41           85615135
 2007_2013 BLANCO  HATCHBACK  Publico 19_41           82913885
 2000_2006 

In [12]:
# Crear tabla resumida por rangos de severidad para LaTeX
tabla_severidad_documentacion <- tabla_severidad_completa %>%
  mutate(
    Rango_Severidad = case_when(
      Severidad_Esperada >= 50000000 ~ "Muy Alta (≥50M)",
      Severidad_Esperada >= 20000000 ~ "Alta (20M-50M)",
      Severidad_Esperada >= 5000000 ~ "Media-Alta (5M-20M)",
      Severidad_Esperada >= 1000000 ~ "Media (1M-5M)",
      Severidad_Esperada >= 500000 ~ "Media-Baja (0.5M-1M)",
      TRUE ~ "Baja (<0.5M)"
    )
  ) %>%
  group_by(Rango_Severidad) %>%
  summarise(
    Cantidad_Combinaciones = n(),
    Severidad_Min = min(Severidad_Esperada),
    Severidad_Max = max(Severidad_Esperada),
    Severidad_Promedio = mean(Severidad_Esperada),
    .groups = "drop"
  ) %>%
  mutate(
    Severidad_Min_Format = paste0("$", format(Severidad_Min, big.mark = ",")),
    Severidad_Max_Format = paste0("$", format(Severidad_Max, big.mark = ",")),
    Severidad_Promedio_Format = paste0("$", format(round(Severidad_Promedio), big.mark = ","))
  )

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("           DISTRIBUCIÓN DE SEVERIDAD POR RANGOS                    \n")
cat("═══════════════════════════════════════════════════════════════════\n")
print(tabla_severidad_documentacion, row.names = FALSE)

# Crear tabla detallada con las primeras 50 combinaciones para LaTeX
tabla_top_50_severidad <- tabla_severidad_completa %>%
  head(50) %>%
  mutate(
    Ranking = 1:50,
    Severidad_LaTeX = paste0("\\$", format(Severidad_Esperada, big.mark = ","))
  ) %>%
  select(Ranking, Modelo, Color, Carroceria, SERVICIO, Edad, Severidad_LaTeX)

cat("\n═══════════════════════════════════════════════════════════════════\n")
cat("           TOP 50 COMBINACIONES DE MAYOR SEVERIDAD                 \n")
cat("═══════════════════════════════════════════════════════════════════\n")
print(tabla_top_50_severidad, row.names = FALSE)

# Exportar tablas para LaTeX
write.csv(tabla_severidad_documentacion, "../data/output/tabla_rangos_severidad_latex.csv", row.names = FALSE)
write.csv(tabla_top_50_severidad, "../data/output/tabla_top_50_severidad_latex.csv", row.names = FALSE)

cat("\n✅ Tablas exportadas para documentación LaTeX:\n")
cat("   - tabla_rangos_severidad_latex.csv\n")
cat("   - tabla_top_50_severidad_latex.csv\n")


═══════════════════════════════════════════════════════════════════
           DISTRIBUCIÓN DE SEVERIDAD POR RANGOS                    
═══════════════════════════════════════════════════════════════════
# A tibble: 6 × 8
  Rango_Severidad      Cantidad_Combinaciones Severidad_Min Severidad_Max
  <chr>                                 <int>         <dbl>         <dbl>
1 Alta (20M-50M)                           91      20071194      49475338
2 Baja (<0.5M)                            110         36772        497926
3 Media (1M-5M)                           252       1011097       4995068
4 Media-Alta (5M-20M)                     213       5010001      19676007
5 Media-Baja (0.5M-1M)                     74        504145        981189
6 Muy Alta (≥50M)                          28      53241001     168558185
# ℹ 4 more variables: Severidad_Promedio <dbl>, Severidad_Min_Format <chr>,
#   Severidad_Max_Format <chr>, Severidad_Promedio_Format <chr>

════════════════════════════════════════════

## Tabla de Severidad para Documentación LaTeX

Crear una tabla más manejable con las 768 combinaciones organizadas para insertar en la documentación.

## Tabla Completa de Severidad para LaTeX

Generar tabla completa con todas las 768 combinaciones de severidad para insertar en documentación.